# Data Collection for Dashboard Visualisation

Google trends data collection for each Places Of Interest (POI) within 6 countries (Indonesia, Japan, Singapore, South Korea, Taiwan, Thailand). Data preparation and insertion to Database for frontend entity visualisation.

Procedure:
    1. Search for POI in Google Trends: https://trends.google.com/trends/?geo=US
    2. Modify timeframe filter and download csv file
    3. Rename file to entity name and place inside the respective countries' folder (in ./google-trends-data)
    4. Run script to pick up data and format for Database insertion

In [1]:
import pandas as pd
import json, os

## File Paths

In [2]:
# Local file paths
fullActivitiesDatasetFilepath = "C:/Users/Admin/OneDrive - National University of Singapore/NUS/NUS Study Materials/NUS Study Materials(2019-2020)/Sem 1/BT3101 - Business Analytics Capstone Project/Activities_Crawling_Project/Scrapy_Tripadvisor/"
fullActivitiesfilename = 'Full_Activities_ZJ.csv'

googleTrendsDatasetFilepath = './google-trends-data/'
filename = 'multiTimeline.csv'
dateEntitiesFilename = 'dateEntitiesDbInsertion.json'
finalDateEntitiesFilename = 'finalDateEntitiesDbInsertion.json'
finalMonetizeEntitiesFilename = 'finalMonetizeEntitiesDbInsertion.json'
timePeriodEntitiesFilename = 'timePeriodEntitiesDbInsertion.json'
finalTimePeriodEntitiesFilename = 'finalTimePeriodEntitiesDbInsertion.json'

In [3]:
listOfCountries = os.listdir(googleTrendsDatasetFilepath) 
listOfCountries

['Indonesia', 'Japan', 'Singapore', 'South Korea', 'Taiwan', 'Thailand']

## Read in Dataset

In [4]:
outputFiles = False # Set to True to write json files

### Format Data (Rach)

In [5]:
monthMapping = {
    '01': '1',
    '02': '2',
    '03': '3',
    '04': '4',
    '05': '5',
    '06': '6',
    '07': '7',
    '08': '8',
    '09': '9',
    '10': '10',
    '11': '11',
    '12': '12'
}

In [6]:
def getMonthAndYear(dateStr):
    month, year = '', ''
    try:
        tempList = dateStr.split('-')
        month = monthMapping.get(tempList[1])
        year = tempList[0]
    except Exception as e:
        print(e)
    finally:
        return month, year

In [7]:
# Input is assumed to have unique month data per entity per country per year 
def addEntityInDict(dateEntitiesDict, country, year, mth, entity, entityCount):
    if country not in dateEntitiesDict.keys():
        dateEntitiesDict.update({
            country: {
                year: {
                    'entities': {
                        entity : [[mth, entityCount]]
                    }
                }
            }
        })
    elif year not in dateEntitiesDict.get(country).keys():
        dateEntitiesDict.get(country).update({
            year: {
                    'entities': {
                        entity : [[mth, entityCount]]
                    }
                }
        })
    elif entity not in dateEntitiesDict.get(country).get(year).get('entities').keys():
#         print(entity)
#         print(mth, year)
#         print(dateEntitiesDict.get(country).get(year).get('entities').keys())
#         print('\n')
        dateEntitiesDict.get(country).get(year).get('entities').update({
            entity : [[mth, entityCount]]
        })
    else:
#         print(entity)
#         print(mth, year)
        entities = dateEntitiesDict.get(country).get(year).get('entities')
        if entities.get(entity):
            updatedEntities = entities.get(entity)
            updatedEntities.append([mth, entityCount])
            entities.update({
                entity : updatedEntities
            })
        else:
            entities.update({
                entity :[[mth, entityCount]]
            })
    return dateEntitiesDict

In [8]:
dateEntitiesDict = {}
for country in listOfCountries:
    listOfEntities = os.listdir(googleTrendsDatasetFilepath + country)
    print(country)
    for entityFilename in listOfEntities:
        trendDf = pd.read_csv(googleTrendsDatasetFilepath + country + '/' + entityFilename, skiprows=1)
        ColumnNames = trendDf.columns
        for index, row in trendDf.iterrows():
            mth, year = getMonthAndYear(row['Month'])
            entityCount = row[ColumnNames[1]]
            entity = entityFilename.split('.csv')[0]
#             print(mth)
#             print(year)
#             print(dateEntitiesDict, country, year, month, entity, entityCount)
            dateEntitiesDict = addEntityInDict(dateEntitiesDict, country, year, mth, entity, entityCount)
# dateEntitiesDict

Indonesia
Japan
Singapore
South Korea
Taiwan
Thailand


In [9]:
# Writes to output json 
if outputFiles:
    with open(outputFilename, 'w') as fp:
        json.dump(dateEntitiesDict, fp)

In [10]:
if dateEntitiesFilename:
    with open(dateEntitiesFilename, 'r') as fp:
        dateEntitiesDict = json.load(fp)

In [11]:
finalDateEntitiesDict = {}
for country in dateEntitiesDict.keys():
    yearEntitiesDict = {}
    for year in dateEntitiesDict.get(country).keys():
        entitiesList = []
        for entity in dateEntitiesDict.get(country).get(year).get('entities').keys():
#             print(entity)
            listOfEntityMonthCount = []
            for entityCountPerMth in dateEntitiesDict.get(country).get(year).get('entities').get(entity):
#                 print(entityCountPerMth)
                listOfEntityMonthCount.append({'x':entityCountPerMth[0], 'y':entityCountPerMth[1]})
#             print(listOfEntityMonthCount)
            entitiesList.append({'name': entity, 'data': listOfEntityMonthCount})
#             print(entitiesDict)
        yearEntitiesDict.update({year: {'entities': entitiesList}})
#         break
    finalDateEntitiesDict.update({country: yearEntitiesDict})
#     break
finalDateEntitiesDict

{'Indonesia': {'2010': {'entities': [{'name': 'Banda Islands',
     'data': [{'x': '1', 'y': 52},
      {'x': '2', 'y': 65},
      {'x': '3', 'y': 53},
      {'x': '4', 'y': 71},
      {'x': '5', 'y': 42},
      {'x': '6', 'y': 76},
      {'x': '7', 'y': 44},
      {'x': '8', 'y': 64},
      {'x': '9', 'y': 45},
      {'x': '10', 'y': 50},
      {'x': '11', 'y': 73},
      {'x': '12', 'y': 15}]},
    {'name': 'Borobudur',
     'data': [{'x': '1', 'y': 43},
      {'x': '2', 'y': 47},
      {'x': '3', 'y': 43},
      {'x': '4', 'y': 45},
      {'x': '5', 'y': 47},
      {'x': '6', 'y': 39},
      {'x': '7', 'y': 46},
      {'x': '8', 'y': 50},
      {'x': '9', 'y': 41},
      {'x': '10', 'y': 51},
      {'x': '11', 'y': 55},
      {'x': '12', 'y': 40}]},
    {'name': 'Bunaken',
     'data': [{'x': '1', 'y': 15},
      {'x': '2', 'y': 16},
      {'x': '3', 'y': 16},
      {'x': '4', 'y': 20},
      {'x': '5', 'y': 17},
      {'x': '6', 'y': 15},
      {'x': '7', 'y': 20},
      {'x': '8',

In [12]:
# Writes to output json 
if outputFiles:
    with open(finalDateEntitiesFilename, 'w') as fp:
        json.dump(finalDateEntitiesDict, fp)

### Format Data for Monetization Entities (Rach)

In [13]:
fullActivitiesDf = pd.read_csv(fullActivitiesDatasetFilepath + fullActivitiesfilename)
fullActivitiesDf

,Country,City,Title,Activities_Type,Review_Count,Rating,Description,Address,Open,Close,Lat,Long,A_Type,Review_Rating
0,Thailand,Phuket,Bang Tao Beach,"Outdoor Activities, Nature & Parks, Beaches",809.0,4.5,NaN,"Bang Tao Beach, Phuket 83110, Thailand",NaN,NaN,8.002546,98.292632,"Active, Water Activities, Nature",3640.5
1,Thailand,"Phuket, Kathu, Patong",Freedom Beach,"Outdoor Activities, Nature & Parks, Beaches",3078.0,4.5,NaN,"Patong, Kathu, Phuket, Thailand",NaN,NaN,7.896576,98.302104,"Active, Water Activities, Nature",13851
2,Thailand,"Phuket, Phuket Town",Monkey Hill,"Nature & Parks, Nature & Wildlife Areas",892.0,4.5,NaN,"To Chae Road, Phuket Town, Phuket, Thailand",NaN,NaN,7.897273,98.398138,Nature,4014
3,Thailand,"Phuket, Kathu, Patong",Jungceylon,"Shopping, Shopping Malls",6264.0,4.0,The Epicenter of Voguish Trend Discover an unr...,"181, Rat-U-Thit 200 Pee Road, Patong, Kathu, P...",1100.0,2200.0,7.891089,98.299898,City,25056
4,Thailand,"Phuket, Kathu, Patong",Bangla Road,"Sights & Landmarks, Points of Interest & Landm...",18040.0,4.0,NaN,"Soi Bangla, Patong, Kathu, Phuket 83150, Thailand",NaN,NaN,7.893583,98.296804,Attractions,72160
5,Thailand,Phuket,Soi Dog Foundation,"Nature & Parks, Nature & Wildlife Areas",941.0,5.0,Soi Dog Foundation is S.E. Asia’s largest resc...,"167/9 Soi Mai Khao 10, Phuket 83110, Thailand",900.0,1530.0,8.127863,98.313793,Nature,4705
6,Thailand,"Phuket, Karon",Karon Beach,"Nature & Parks, Outdoor Activities, Beaches",6170.0,4.0,"As featured in <a href=""/Guide-g297930-k1023-P...","Karon, Phuket, Thailand",NaN,NaN,7.828774,98.301286,"Active, Water Activities, Nature",24680
7,Thailand,"Phuket, Kathu, Patong",Patong Beach,"Outdoor Activities, Nature & Parks, Sights & L...",14606.0,3.5,NaN,"Patong, Kathu, Phuket 83000, Thailand",NaN,NaN,7.896576,98.302104,"Food, Active, Nightlife, Water Activities, Nat...",51121
8,Thailand,"Phuket, Phuket Town, Chalong",Big Buddha Phuket,"Sights & Landmarks, Sacred & Religious Sites, ...",14007.0,4.5,NaN,"Soi Yot Sane 1, Chaofa West Road, Chalong, Phu...",800.0,1930.0,7.832293,98.336766,"Cultural, Attractions",63031.5
9,Thailand,"Phuket, Karon, Kata Beach",Kata Beach,"Outdoor Activities, Nature & Parks, Beaches",7141.0,4.0,"As featured in <a href=""/Guide-g293920-k1016-P...","Pakbang Road, Kata Beach, Karon, Phuket 83100,...",NaN,NaN,7.820337,98.298125,"Active, Water Activities, Nature",28564


In [14]:
selectedActivities = fullActivitiesDf[['Country', 'City', 'Title']]
selectedActivities

,Country,City,Title
0,Thailand,Phuket,Bang Tao Beach
1,Thailand,"Phuket, Kathu, Patong",Freedom Beach
2,Thailand,"Phuket, Phuket Town",Monkey Hill
3,Thailand,"Phuket, Kathu, Patong",Jungceylon
4,Thailand,"Phuket, Kathu, Patong",Bangla Road
5,Thailand,Phuket,Soi Dog Foundation
6,Thailand,"Phuket, Karon",Karon Beach
7,Thailand,"Phuket, Kathu, Patong",Patong Beach
8,Thailand,"Phuket, Phuket Town, Chalong",Big Buddha Phuket
9,Thailand,"Phuket, Karon, Kata Beach",Kata Beach


In [35]:
monetizeEntitiesList = []
monetizeEntitiesDict = {}
countryEntitiesDict = {}
entityList = []
for index, row in selectedActivities.iterrows():
    country = row['Country']
    if country not in countryEntitiesDict.keys():
        countryEntitiesDict.update({country: [{'entity': row['Title'], 'city': row['City']}]}) 
    else:
        countryEntitiesDict.get(country).append({'entity': row['Title'], 'city': row['City']})
# print(countryEntitiesDict)
for country, data in countryEntitiesDict.items():
    monetizeEntitiesList.append({'country': country, 'data': data})
    monetizeEntitiesDict = {'monetizeEntities': monetizeEntitiesList}
monetizeEntitiesDict

{'monetizeEntities': [{'country': 'Thailand',
   'data': [{'entity': 'Bang Tao Beach', 'city': 'Phuket'},
    {'entity': 'Freedom Beach', 'city': 'Phuket, Kathu, Patong'},
    {'entity': 'Monkey Hill', 'city': 'Phuket, Phuket Town'},
    {'entity': 'Jungceylon', 'city': 'Phuket, Kathu, Patong'},
    {'entity': 'Bangla Road', 'city': 'Phuket, Kathu, Patong'},
    {'entity': 'Soi Dog Foundation', 'city': 'Phuket'},
    {'entity': 'Karon Beach', 'city': 'Phuket, Karon'},
    {'entity': 'Patong Beach', 'city': 'Phuket, Kathu, Patong'},
    {'entity': 'Big Buddha Phuket', 'city': 'Phuket, Phuket Town, Chalong'},
    {'entity': 'Kata Beach', 'city': 'Phuket, Karon, Kata Beach'},
    {'entity': 'Mai Khao Beach', 'city': 'Phuket'},
    {'entity': 'Patong Boxing Stadium', 'city': 'Phuket, Kathu, Patong'},
    {'entity': 'Karon View Point', 'city': 'Phuket, Karon'},
    {'entity': 'Kata Noi Beach', 'city': 'Phuket, Karon'},
    {'entity': 'Tiger Kingdom Phuket', 'city': 'Phuket, Kathu'},
    {'e

In [37]:
# Writes to output json 
if outputFiles:
    with open(finalMonetizeEntitiesFilename, 'w') as fp:
        json.dump(monetizeEntitiesDict, fp)

### Format Data (Des)

In [22]:
if timePeriodEntitiesFilename:
    with open(timePeriodEntitiesFilename, 'r') as fp:
        timePeriodEntitiesDict = json.load(fp)
timePeriodEntitiesDict

{'oneDay': {'entities': {'Asian Civilisations Museum': 1,
   'Osaka': 1,
   'Marina Bay': 1,
   'Maxwell Food Centre': 1,
   'Merlion Park': 1,
   'National Museum of Singapore': 1,
   'Night Safari': 1,
   'Orchard Road': 1,
   'River Safari': 1,
   'Bollywood Veggies': 1,
   'Sultan Mosque': 1,
   'Sungei Buloh Wetland Reserve': 1,
   'Supertree Grove': 1,
   'Tanjong Pagar': 1,
   'The Intan': 1,
   'UOB Plaza': 1,
   'Marina Barrage': 1,
   'Malay Heritage Centre': 1,
   'MacRitchie Reservoir': 1,
   'Little India': 1,
   'Kusu Island': 1,
   'Kranji Countryside': 1,
   'Jurong Bird Park': 1,
   'Indian Heritage Centre': 1,
   'ION Orchard': 1,
   'Hay Dairies': 1,
   'Haji Lane': 1,
   'Golden Mile Tower': 1,
   'Giant Swing': 1,
   'Gardens by the Bay': 1,
   'Chinatown': 1,
   'Chijmes': 1,
   'Brand New': 1,
   'Woodlands': 1}},
 'sevenDays': {'entities': {'Asian Civilisations Museum': 1,
   'Osaka': 1,
   'Marina Bay': 1,
   'Maxwell Food Centre': 1,
   'Merlion Park': 1,
   '

In [32]:
from random import randint
for timePeriod, entities in timePeriodEntitiesDict.items():
    for entity, entityCount in entities.get('entities').items():
        timePeriodEntitiesDict.get(timePeriod).get('entities')[entity] = randint(0,100)
timePeriodEntitiesDict

{'oneDay': {'entities': {'Asian Civilisations Museum': 38,
   'Osaka': 63,
   'Marina Bay': 41,
   'Maxwell Food Centre': 55,
   'Merlion Park': 56,
   'National Museum of Singapore': 68,
   'Night Safari': 47,
   'Orchard Road': 15,
   'River Safari': 67,
   'Bollywood Veggies': 86,
   'Sultan Mosque': 83,
   'Sungei Buloh Wetland Reserve': 91,
   'Supertree Grove': 51,
   'Tanjong Pagar': 47,
   'The Intan': 77,
   'UOB Plaza': 52,
   'Marina Barrage': 79,
   'Malay Heritage Centre': 30,
   'MacRitchie Reservoir': 78,
   'Little India': 22,
   'Kusu Island': 40,
   'Kranji Countryside': 74,
   'Jurong Bird Park': 30,
   'Indian Heritage Centre': 80,
   'ION Orchard': 45,
   'Hay Dairies': 79,
   'Haji Lane': 36,
   'Golden Mile Tower': 8,
   'Giant Swing': 35,
   'Gardens by the Bay': 84,
   'Chinatown': 75,
   'Chijmes': 40,
   'Brand New': 98,
   'Woodlands': 18}},
 'sevenDays': {'entities': {'Asian Civilisations Museum': 0,
   'Osaka': 21,
   'Marina Bay': 29,
   'Maxwell Food Cen

In [33]:
# Writes to output json 
# if outputFiles:
with open(finalTimePeriodEntitiesFilename, 'w') as fp:
    json.dump(timePeriodEntitiesDict, fp)